# **Aerial wifi reconnaissance**

In [ ]:
# import dataset

import pandas as pd
wd = pd.read_csv ('./total_data.csv', delimiter = ',')  # read warflight csv file into Pandas dataframe

print("Total WiFi AP entries: " + str(len(wd)))
wd.sample(10)  # 10 random networks

Total WiFi AP entries: 918


,MAC,SSID,Encryption,FirstSeen,Channel,RSSI,Latitude,Longitude,AltitudeMeters,AccuracyMeters,Type
292,5C:02:14:C6:CE:BB,SONA_DVR2,[WPA2-PSK-CCMP+TKIP][ESS],8/21/2021 18:38,11,-89,30.410882,77.968841,964.0,235,WIFI
755,2A:E8:29:5A:1B:8C,MySpectrumWiFi90-2G,[WPA2-PSK-CCMP+TKIP][ESS],8/21/2021 18:39,7,-84,30.407709,77.969970,963.7,239,WIFI
419,10:FE:ED:58:87:D8,MEGAHERTZ_102,[WPA2-PSK-CCMP+TKIP][ESS],8/21/2021 18:38,2,-91,30.412725,77.971748,960.1,236,WIFI
221,50:2B:73:D8:18:B0,"Doon Marvel,",[WPA2-PSK-CCMP+TKIP][ESS],8/21/2021 18:37,1,-89,30.411128,77.966842,967.6,234,WIFI
6,1C:18:4A:84:4A:60,MB Deathawk,[WPA2-PSK-CCMP+TKIP][ESS],8/21/2021 18:36,9,-86,30.411178,77.967056,967.7,290,WIFI
812,10:DA:43:80:A3:D1,Logjam - private,[WPA2-PSK-CCMP+TKIP][ESS],8/21/2021 18:39,11,-88,30.410596,77.968452,963.8,239,WIFI
491,54:47:E8:0D:82:95,Missoula Paddleheads - Private,[WPA2-PSK-CCMP+TKIP][ESS],8/21/2021 18:38,11,-90,30.408716,77.970077,968.3,294,WIFI
402,7C:A9:6B:8C:87:4D,POCO M3,[WPA2-PSK-CCMP+TKIP][ESS],8/21/2021 18:38,6,-88,30.412904,77.971641,960.1,236,WIFI
888,74:D0:2B:D1:15:48,Pov Staff Only,[WPA2-PSK-CCMP+TKIP][ESS],8/21/2021 18:40,11,-67,30.411123,77.967010,964.5,240,WIFI
51,04:95:E6:41:90:48,Tenda_419048,[ESS],8/21/2021 18:37,3,-83,30.411304,77.967033,969.6,291,WIFI


In [ ]:
# clean dataset by dropping excessive data entries

wd.drop(wd.index[wd["FirstSeen"] <= "8/21/2021 18:36"],0, inplace=True) # before takeoff
wd.drop(wd.index[wd["FirstSeen"] >= "8/21/2021 18:40"],0, inplace=True) # after landing

print("Total WiFi AP entries: " + str(len(wd)))
print("Unique network entries: " + str(len(wd.SSID.unique())))

Total WiFi AP entries: 809
Unique network entries: 252


<ipython-input-24-d683bd1dc822>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  wd.drop(wd.index[wd["FirstSeen"] <= "8/21/2021 18:36"],0, inplace=True) # before takeoff
<ipython-input-24-d683bd1dc822>:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  wd.drop(wd.index[wd["FirstSeen"] >= "8/21/2021 18:40"],0, inplace=True) # after landing


# **Map All Data Entries + WarFlying Route**

In [ ]:
# plot all network entries as a cluster map w/ warflight route

import folium
from folium import FeatureGroup, LayerControl, Map, Marker
from folium.plugins import MarkerCluster

MarkerCluster()

# autofit map boundaries from dataset
wd_net_map = folium.Map(wd[['Latitude', 'Longitude']].mean().values.tolist())
wd_net_map.fit_bounds([wd[['Latitude', 'Longitude']].min().values.tolist(), wd[['Latitude', 'Longitude']].max().values.tolist()])
network_cluster = MarkerCluster().add_to(wd_net_map)

# convert dataframe to array
networks = wd.values

# warflight route
folium.PolyLine(wd[['Latitude', 'Longitude']].values.tolist(),line_opacity = 0.5, weight = 4).add_to(wd_net_map)

# cluster map of all network entries
for network in networks: folium.Marker([network[6],network[7]], popup=network[1], icon=folium.Icon(color='darkblue', icon_color='white', icon='wifi', angle=0, prefix='fa')).add_to(network_cluster)

# first and last data entry
folium.Marker([networks[0][6], networks[0][7]], popup=networks[0][3], icon=folium.Icon(color='green', icon_color='white', icon='plane', angle=0, prefix='fa')).add_to(wd_net_map)
folium.Marker([networks[len(networks)-1][6],networks[len(networks)-1][7]], popup= networks[len(networks)-1][3], icon=folium.Icon(color='red', icon_color='white', icon='plane', angle=0, prefix='fa')).add_to(wd_net_map)

display(wd_net_map)

# **Known Devices + Network HeatMap**

In [ ]:
# network concentration heat map + known device location

known_devices = ["Target3", "Bingus Mobile"]

import folium, numpy
from folium import Map, Marker, plugins

# autofit map boundaries from dataset
wd_heat_map = folium.Map(wd[['Latitude', 'Longitude']].mean().values.tolist())
wd_heat_map.fit_bounds([wd[['Latitude', 'Longitude']].min().values.tolist(), wd[['Latitude', 'Longitude']].max().values.tolist()])

for device in wd.loc[wd["SSID"].isin(known_devices)].values:
  folium.Marker( location=[device[6], device[7]], popup=device[1], icon=folium.Icon(color="darkblue",icon='wifi',prefix='fa')).add_to(wd_heat_map)

wd_heat_map.add_child(plugins.HeatMap(wd[['Latitude', 'Longitude']].to_numpy(), radius=30))
display(wd_heat_map)

# **WiFi Network Encryption Filter**

This feature map lets you view and toggle WiFi networks by encryption type.  There were a surprising amount of open / unsecured networks! 

In [ ]:
import folium
from folium import FeatureGroup, LayerControl, Map, Marker

wd_encryption_map = folium.Map(wd[['Latitude', 'Longitude']].mean().values.tolist())
wd_encryption_map.fit_bounds([wd[['Latitude', 'Longitude']].min().values.tolist(), wd[['Latitude', 'Longitude']].max().values.tolist()])

# map feature groups for different encryption types
wep_feature_group = folium.FeatureGroup("WEP")
wpa_feature_group = folium.FeatureGroup("WPA")
wpa2_feature_group = folium.FeatureGroup("WPA2")
opn_feature_group = folium.FeatureGroup("Open")
none_feature_group = folium.FeatureGroup("None")

# convert dataframe to array
networks = wd.values

# add network coordinates
for network in networks:
  if   (network[2] == "[WPA-PSK-CCMP+TKIP][ESS]"):
    folium.Marker(location=[network[6],network[7]],  popup=network[1], icon=folium.Icon(color="blue",icon='user-secret',prefix='fa')).add_to(wpa_feature_group)

  elif (network[2] == "[WPA2-PSK-CCMP+TKIP][ESS]"):
    folium.Marker(location=[network[6],network[7]],  popup=network[1], icon=folium.Icon(color="green",icon='user-secret',prefix='fa')).add_to(wpa2_feature_group)

  elif (network[2] == "[WEP][ESS]"):
    folium.Marker(location=[network[6],network[7]],  popup=network[1], icon=folium.Icon(color="red",icon='user-secret',prefix='fa')).add_to(wep_feature_group)

  elif (network[2] == "[ESS]"):
    folium.Marker(location=[network[6],network[7]],  popup=network[1], icon=folium.Icon(color="orange",icon='user-secret',prefix='fa')).add_to(opn_feature_group)

  elif (network[2] == "[WPA-PSK-CCMP+TKIP][WPA2-PSK-CCMP+TKIP][ESS]"):
    folium.Marker(location=[network[6],network[7]],  popup=network[1], icon=folium.Icon(color="purple",icon='user-secret',prefix='fa')).add_to(none_feature_group)

# add feature groups to map 
wep_feature_group.add_to(wd_encryption_map)
wpa_feature_group.add_to(wd_encryption_map)
wpa2_feature_group.add_to(wd_encryption_map)
opn_feature_group.add_to(wd_encryption_map)
none_feature_group.add_to(wd_encryption_map)

folium.LayerControl().add_to(wd_encryption_map)

display(wd_encryption_map)